# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import warnings
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


warnings.filterwarnings('ignore')

Загрузим данные геологоразведки трёх регионов:

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [3]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
df_2 = pd.read_csv('/datasets/geo_data_2.csv')
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


+ id — уникальный идентификатор скважины;
+ f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
+ product — объём запасов в скважине (тыс. баррелей).

Во всех таблицах отсутствуют пропуски. Данные готовы для разделения на выборки и последующего обучения. 

Удалим идентификатор из каждой таблицы, т.к они только нагружают модель и никакого значеия не несут. 

In [8]:
df_0 = df_0.drop(['id'], axis=1)
df_1 = df_1.drop(['id'], axis=1)
df_2 = df_2.drop(['id'], axis=1)

## Обучение и проверка модели

### Разбивка данных на обучающую и валидационную выборки. 

Подготовим функцию для разбивки данных на обучающие и валидационные выборки(75:25).

In [9]:
def split_data(df, target):
    features = df.drop([target] , axis=1)
    target = df[target]

    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    return features_train, features_valid, target_train, target_valid

Разобьем данные для каждой таблицы на обучающую и валидационную выборки в соотношении 75:25.

In [10]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = split_data(df_0, 'product')

In [11]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(df_1, 'product')

In [12]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(df_2, 'product')

### Обучение модели и предсказания на валидационной выборке.

**Убедимся в необходимости масштабирования признаков.**

In [13]:
model = LinearRegression()

model.fit(features_train_0, target_train_0)
predicted_target_0 = model.predict(features_valid_0)
print('На немастшабированных данных RMSE -', mean_squared_error(target_valid_0, predicted_target_0)**0.5)


На немастшабированных данных RMSE - 37.5794217150813


In [14]:
features_train_0s, features_valid_0s, target_train_0s, target_valid_0s = split_data(df_0, 'product')

numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_0s[numeric])
features_train_0s[numeric] = scaler.transform(features_train_0s[numeric])
features_valid_0s[numeric] = scaler.transform(features_valid_0s[numeric])

model.fit(features_train_0s, target_train_0s)
predicted_target_0s = model.predict(features_valid_0s)
print('На мастшабированных данных RMSE -', mean_squared_error(target_valid_0s, predicted_target_0s)**0.5)


На мастшабированных данных RMSE - 37.5794217150813


**Вывод**: в масташиборовании признаков модели нет необходимости, т.к обе модели показали одинаоквые метрики RMSE - 37.5794217150813

**Подготовим функцию для обучения и предсказания модели.**

In [15]:
def model_prediction(df, features_train, features_valid, target_train, target_valid):
    
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse**0.5
    mean_product = predicted_valid.mean()

    return rmse, mean_product, predicted_valid


### Средний запас предсказанного сырья и RMSE модели.

In [16]:
rmse_0, mean_product_0, predicted_valid_0 = model_prediction(df_0, features_train_0, features_valid_0, target_train_0, target_valid_0)
print(f'Регион №1. Средний запас предсказанного сырья: {mean_product_0}, RMSE = {rmse_0}')


Регион №1. Средний запас предсказанного сырья: 92.59256778438038, RMSE = 37.5794217150813


In [17]:
rmse_1, mean_product_1, predicted_valid_1 = model_prediction(df_1, features_train_1, features_valid_1, target_train_1, target_valid_1)
print(f'Регион №2. Средний запас предсказанного сырья: {mean_product_1}, RMSE = {rmse_1}')


Регион №2. Средний запас предсказанного сырья: 68.728546895446, RMSE = 0.893099286775616


In [18]:
rmse_2, mean_product_2, predicted_valid_2 = model_prediction(df_2, features_train_2, features_valid_2, target_train_2, target_valid_2)
print(f'Регион №3. Средний запас предсказанного сырья: {mean_product_2}, RMSE = {rmse_2}')

Регион №3. Средний запас предсказанного сырья: 94.96504596800489, RMSE = 40.02970873393434


**Вывод:** для дальнейшего рассмотрения больше всего подходят 1 и 3 регионы. В данных регионах находится наибольшее количества сырья. 

## Подготовка к расчёту прибыли

**Исходные данные для расчета прибыли.**

In [19]:
budget = 10000000000
all_wells = 500
best_wells = 200
price_per_barrel = 450

**Объём сырья для безубыточной разработки новой скважины**

In [20]:
raw_material_volume = (budget/price_per_barrel)/best_wells
print(f'Объём сырья для безубыточной разработки новой скважины - {raw_material_volume} барелей')

Объём сырья для безубыточной разработки новой скважины - 111111.11111111112 барелей


In [21]:
print(f'Регион №1. Средний запас предсказанного сырья: {mean_product_0}')
print(f'Разница объёма сырья для безубыточной разработки новой скважины и предсказанного объема сырья составляет:') 
print()      
print(f'{mean_product_0 * 1000-raw_material_volume} барелей')

Регион №1. Средний запас предсказанного сырья: 92.59256778438038
Разница объёма сырья для безубыточной разработки новой скважины и предсказанного объема сырья составляет:

-18518.543326730753 барелей


In [22]:
print(f'Регион №2. Средний запас предсказанного сырья: {mean_product_1}')
print(f'Разница объёма сырья для безубыточной разработки новой скважины и предсказанного объема сырья составляет:') 
print()      
print(f'{mean_product_1 * 1000-raw_material_volume} барелей')

Регион №2. Средний запас предсказанного сырья: 68.728546895446
Разница объёма сырья для безубыточной разработки новой скважины и предсказанного объема сырья составляет:

-42382.564215665116 барелей


In [23]:
print(f'Регион №3. Средний запас предсказанного сырья: {mean_product_2}')
print(f'Разница объёма сырья для безубыточной разработки новой скважины и предсказанного объема сырья составляет:') 
print()      
print(f'{mean_product_2 * 1000-raw_material_volume} барелей')

Регион №3. Средний запас предсказанного сырья: 94.96504596800489
Разница объёма сырья для безубыточной разработки новой скважины и предсказанного объема сырья составляет:

-16146.065143106243 барелей


**Наименьшая разница объема сырья между безубыточной разработкой и предсказанным объемом составляет у 3 региона, наибольшая - 2 регион**

## Расчёт прибыли и рисков 

Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [24]:
def revenue(target, predictions, count):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    predictions_s = predictions.sort_values(ascending=False)[:count].index
    selected = target[predictions_s]
    return price_per_barrel*1000 * selected.sum() - budget


In [25]:
def bootstrap(target, probabilities, iterations, sample_size):
    state = np.random.RandomState(12345)
    values = []
    for i in range(iterations):
        target_subsample = target.sample(n=sample_size, random_state=state, replace=True)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, best_wells))

    values = pd.Series(values)
    mean = values.mean()
    risk = (values < 0).mean()
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return round(mean,2), round(risk, 2), lower, upper

In [26]:
mean_0, risk_0, lower_0, upper_0 = bootstrap(target_valid_0.reset_index(drop=True), predicted_valid_0, 1000, 500)
print('Данные по первому региону.')
print('Средняя прибыль по региону = {:.2f} тыс.руб.'.format(mean_0))
print('95% доверительный интервал от {:.2f} до {:.2f} тыс.руб.'.format(lower_0, upper_0))
print('Процент риска {:.2%}'.format(risk_0))

Данные по первому региону.
Средняя прибыль по региону = 396164984.80 тыс.руб.
95% доверительный интервал от -111215545.89 до 909766941.55 тыс.руб.
Процент риска 7.00%


In [27]:
mean_1, risk_1, lower_1, upper_1 = bootstrap(target_valid_1.reset_index(drop=True), predicted_valid_1, 1000, 500)
print('Данные по второму региону.')
print('Средняя прибыль по региону = {:.2f} руб.'.format(mean_1))
print('95% доверительный интервал от {:.2f} до {:.2f} руб.'.format(lower_1, upper_1))
print('Процент риска {:.2%}'.format(risk_1))

Данные по второму региону.
Средняя прибыль по региону = 456045105.79 руб.
95% доверительный интервал от 33820509.40 до 852289453.87 руб.
Процент риска 2.00%


In [28]:
mean_2, risk_2, lower_2, upper_2 = bootstrap(target_valid_2.reset_index(drop=True), predicted_valid_2, 1000, 500)
print('Данные по второму региону.')
print('Средняя прибыль по региону = {:.2f} руб.'.format(mean_2))
print('95% доверительный интервал от {:.2f} до {:.2f} руб.'.format(lower_2, upper_2))
print('Процент риска {:.2%}'.format(risk_2))

Данные по второму региону.
Средняя прибыль по региону = 404403866.57 руб.
95% доверительный интервал от -163350413.40 до 950359574.92 руб.
Процент риска 8.00%


***Вывод***

Согласно прогнозам сделанным с помощью техники Bootstrap, наиболее прибыльным будет разработка месторождений в регионе №2. В данном регионе самые высокие показатели средней выручки и 95%-ного доверительного интервала. Также в регионе №2 самый низкий риск убытков.